# Projecting Food Insecurity Rates in the US by County
## Secondary Data Cleaning
The following process imports a cleaned dataset produced from cleaning_pt1.ipynb. This notebook is used for additional cleaning, such as adjusting datatypes, imputing missing data, and removing/averaging duplicate observations.
### Flatiron School Data Science Capstone<br>By Khyatee Desai

In [9]:
import pandas as pd
import numpy as np
import os
pd.set_option('display.max_columns', None)
import warnings
import pickle
warnings.filterwarnings('ignore')

### Import dataset produced from first cleaning notebook


In [10]:
with open('../pickled/partially_cleaned_data.pickle', "rb") as input_file:
    df = pickle.load(input_file) 
    
#  drop un needed features
df.drop(['FIPS_state', 'FIPS_county'], axis=1, inplace=True)
 

# Unemployment Dataset: Fix Datatypes
Unemployment data was saved as a string due to the presence of special characters "-". The following cell drops these characters, converts the column to a float datatype, then adds it back into the main df
### remove special characters '-' and change numeric datatypes to floats

In [13]:
# Total Workforce Column
df_workf = df[['FIPS','Year','Total_workforce']]
df_workf = df_workf.dropna()
df_workf.drop(df_workf[df_workf['Total_workforce'].values == '–'].index,inplace=True)
df_workf = df_workf['Total_workforce'].astype(float)

# Employed column
df_employed = df[['FIPS','Year','Employed']]
df_employed = df_employed.dropna()
df_employed.drop(df_employed[df_employed['Employed'].values == '–'].index,inplace=True)
df_employed = df_employed['Employed'].astype(float)

# Unemployed column
df_unemployed = df[['FIPS','Year','Unemployed']]
df_unemployed = df_unemployed.dropna()
df_unemployed.drop(df_unemployed[df_unemployed['Unemployed'].values == '–'].index,inplace=True)
df_unemployed = df_unemployed['Unemployed'].astype(float)

# Unemployment Rate column
df_unemploy_rate = df[['FIPS','Year','Unemployment_rate']]
df_unemploy_rate = df_unemploy_rate.dropna()
df_unemploy_rate.drop(df_unemploy_rate[df_unemploy_rate['Unemployment_rate'].values == '–'].index,inplace=True)
df_unemploy_rate = df_unemploy_rate['Unemployment_rate'].astype(float)


### Drop old version from main df, and add float versions back in

In [14]:
df = df.drop(['Total_workforce','Employed','Unemployed','Unemployment_rate'],axis=1)
df = pd.concat([df,df_workf,df_employed,df_unemployed,df_unemploy_rate],axis=1)

In [15]:
df.dtypes

FIPS                    object
Rent                   float64
Year                    object
coc_number              object
Houseless_rate         float64
Sheltered_rate         float64
Unsheltered_rate       float64
hh_med_income          float64
pop_disabled           float64
pop_hs_grad            float64
pop_bachelors          float64
pop_grad_degree        float64
pop_priv_health        float64
pop_public_health      float64
pop_no_health          float64
pop_total              float64
percent_hh_poverty     float64
hh_avg_size            float64
pop_65+                float64
hh_no_vehicle          float64
num_hh                 float64
pop_non_citizen        float64
hh_SNAP                float64
State                   object
County                  object
TOT_POP                float64
TOT_MALE               float64
TOT_FEMALE             float64
TOT_WHITE              float64
TOT_BLACK              float64
TOT_NATIVE             float64
TOT_ASIAN              float64
TOT_PACI

# Feeding America Dataset: Impute Missing Values
The Feeding America datasets on food insecurity only contain data at the State level for the years 2011, 2012, and 2013, however County-level data is needed for mapping, and to have enough observations for modeling. The following process uses data from 2010 and 2014 to calculate average yearly change per county, and then fills in the missing values.
### Isolate rows from main df with missing values for Feeding America data, as well as 2010 and 2014, which will be used to calculate average yearly change

In [16]:
# create separate dataframe for each year, including features that need to be added
fi_10 = df[df.Year=='2010'][['FIPS','FI Rate','Cost Per Meal']]
fi_11 = df[df.Year=='2011'][['FIPS','FI Rate','Cost Per Meal']]
fi_12 = df[df.Year=='2012'][['FIPS','FI Rate','Cost Per Meal']]
fi_13 = df[df.Year=='2013'][['FIPS','FI Rate','Cost Per Meal']]
fi_14 = df[df.Year=='2014'][['FIPS','FI Rate','Cost Per Meal']]


### Merge together 2010 and 2014 dataframes

In [17]:
# merge and rename columns for interpretability
fi_10_and_14 = fi_10.merge(fi_14, on='FIPS',how='inner')
fi_10_and_14.rename(columns={'FI Rate_x':'FI Rate 10', 'Cost Per Meal_x': 'Meal cost 10', 
                             'FI Rate_y':'FI Rate 14', 'Cost Per Meal_y': 'Meal cost 14'}, inplace=True)
fi_10_and_14

,FIPS,FI Rate 10,Meal cost 10,FI Rate 14,Meal cost 14
0,02020,0.123,2.64,0.127,3.30
1,02020,0.123,2.64,0.127,3.30
2,02020,0.123,2.64,0.127,3.30
3,02013,0.145,2.69,0.143,4.13
4,02013,0.145,2.69,0.143,4.13
...,...,...,...,...,...
10399,25999,NaN,NaN,NaN,NaN
10400,26999,NaN,NaN,NaN,NaN
10401,02999,NaN,NaN,NaN,NaN
10402,04999,NaN,NaN,NaN,NaN


### Impute missing values for each year
This cell finds the difference between 2014 and 2010, and add 1/4 of the difference consecutively for each year with missing data.

In [18]:
# calculate difference between 2014 and 2010 for each feature of interest
fi_10_and_14['Rate_diff'] = fi_10_and_14['FI Rate 14'] - fi_10_and_14['FI Rate 10']
fi_10_and_14['Meal_cost_dif'] = fi_10_and_14['Meal cost 14'] - fi_10_and_14['Meal cost 10']

# fill in the missing values for each feature and each of the three years
fi_10_and_14['FI Rate 11'] = fi_10_and_14['FI Rate 10'] + fi_10_and_14['Rate_diff']/4
fi_10_and_14['FI Rate 12'] = fi_10_and_14['FI Rate 11'] + fi_10_and_14['Rate_diff']/4
fi_10_and_14['FI Rate 13'] = fi_10_and_14['FI Rate 12'] + fi_10_and_14['Rate_diff']/4

fi_10_and_14['Meal cost 11'] = fi_10_and_14['Meal cost 10'] + fi_10_and_14['Meal_cost_dif']/4
fi_10_and_14['Meal cost 12'] = fi_10_and_14['Meal cost 11'] + fi_10_and_14['Meal_cost_dif']/4
fi_10_and_14['Meal cost 13'] = fi_10_and_14['Meal cost 12'] + fi_10_and_14['Meal_cost_dif']/4

fi_10_and_14.drop_duplicates(inplace=True)


### Create new df for each year with newly created values

In [19]:
df_11 = df[df.Year=='2011']
df_11 = df_11.merge(fi_10_and_14[['FIPS','FI Rate 11','Meal cost 11']], on='FIPS')
df_11.drop(['FI Rate','Cost Per Meal'],axis=1,inplace=True)
df_11.rename(columns={'FI Rate 11':'FI Rate','Meal cost 11':'Cost Per Meal'},inplace=True)


In [20]:
df_12 = df[df.Year=='2012']
df_12 = df_12.merge(fi_10_and_14[['FIPS','FI Rate 12','Meal cost 12']], on='FIPS')
df_12.drop(['FI Rate','Cost Per Meal'],axis=1,inplace=True)
df_12.rename(columns={'FI Rate 12':'FI Rate','Meal cost 12':'Cost Per Meal'},inplace=True)


In [21]:
df_13 = df[df.Year=='2013']
df_13 = df_13.merge(fi_10_and_14[['FIPS','FI Rate 13','Meal cost 13']], on='FIPS')
df_13.drop(['FI Rate','Cost Per Meal'],axis=1,inplace=True)
df_13.rename(columns={'FI Rate 13':'FI Rate','Meal cost 13':'Cost Per Meal'},inplace=True)


### Concatenate new dfs back with main df

In [29]:
df_11.isnull().sum()

FIPS                      0
Rent                   3375
Year                      0
coc_number              202
Houseless_rate          202
Sheltered_rate          202
Unsheltered_rate        202
hh_med_income            95
pop_disabled            173
pop_hs_grad             173
pop_bachelors           173
pop_grad_degree         173
pop_priv_health          95
pop_public_health        95
pop_no_health            95
pop_total               173
percent_hh_poverty       95
hh_avg_size             173
pop_65+                 173
hh_no_vehicle            95
num_hh                  173
pop_non_citizen         173
hh_SNAP                  95
State                   173
County                  173
TOT_POP                 173
TOT_MALE                173
TOT_FEMALE              173
TOT_WHITE               173
TOT_BLACK               173
TOT_NATIVE              173
TOT_ASIAN               173
TOT_PACIFIC             173
TOT_LATINX              173
State/County             96
Low Threshold Type  

In [22]:
# drop older missing data from main df
df_trimmed = df.drop(df[df.Year.isin(['2011', '2012','2013'])].index)

# add new data
df = pd.concat([df_trimmed, df_11, df_12, df_13])
df

,FIPS,Rent,Year,coc_number,Houseless_rate,Sheltered_rate,Unsheltered_rate,hh_med_income,pop_disabled,pop_hs_grad,pop_bachelors,pop_grad_degree,pop_priv_health,pop_public_health,pop_no_health,pop_total,percent_hh_poverty,hh_avg_size,pop_65+,hh_no_vehicle,num_hh,pop_non_citizen,hh_SNAP,State,County,TOT_POP,TOT_MALE,TOT_FEMALE,TOT_WHITE,TOT_BLACK,TOT_NATIVE,TOT_ASIAN,TOT_PACIFIC,TOT_LATINX,State/County,FI Rate,Low Threshold Type,High Threshold Type,Cost Per Meal,Num_wholesale,Num_restaraunts,Num_grocery,Total_workforce,Employed,Unemployed,Unemployment_rate
0,01073,1019.995960,2014,AL-500,0.001396,0.000961,0.000435,45239.0,94584.0,117854.0,81626.0,52774.0,431638.0,211570.0,81336.0,658834.0,14.8,2.48,87036.0,93630.0,259397.0,17519.0,39967.0,Alabama,Jefferson County,659972.0,312491.0,347481.0,355635.0,284082.0,2029.0,10378.0,347.0,24099.0,"Jefferson County, AL",0.197,SNAP,Other Nutrition Program,2.93,483.0,2693.0,400.0,312131.0,292505.0,19626.0,6.3
1,01117,1229.755051,2014,AL-500,0.001396,0.000961,0.000435,69723.0,22792.0,28911.0,35773.0,18511.0,159655.0,42429.0,19175.0,201168.0,6.2,2.65,23404.0,19762.0,74790.0,7624.0,4706.0,Alabama,Shelby County,206280.0,100304.0,105976.0,174094.0,24247.0,805.0,4403.0,101.0,11872.0,"Shelby County, AL",0.105,SNAP,Other Nutrition Program,3.37,1.0,743.0,2706.0,107208.0,102400.0,4808.0,4.5
2,04003,1051.250000,2014,AZ-500,0.001469,0.000826,0.000643,45974.0,20341.0,21109.0,12968.0,7566.0,76099.0,50498.0,14868.0,130807.0,13.1,2.47,23593.0,16328.0,48846.0,7947.0,7812.0,Arizona,Cochise County,127314.0,64661.0,62653.0,112238.0,5737.0,2165.0,2757.0,510.0,44374.0,"Cochise County, AZ",0.161,"SNAP, Other Nutrition Programs","SNAP, Other Nutrition Programs",2.81,800.0,72.0,340.0,50969.0,46682.0,4287.0,8.4
3,04013,1095.670228,2014,AZ-502,0.001318,0.001084,0.000235,53689.0,399455.0,593094.0,490927.0,273108.0,2444443.0,1204681.0,646167.0,3947382.0,12.7,2.74,507428.0,546028.0,1424244.0,373532.0,171581.0,Arizona,Maricopa County,4093648.0,2024659.0,2068989.0,3449404.0,235660.0,112383.0,172425.0,11190.0,1239835.0,"Maricopa County, AZ",0.158,"SNAP, Other Nutrition Programs","SNAP, Other Nutrition Programs",2.90,2389.0,16857.0,6320.0,1961997.0,1848119.0,113878.0,5.8
4,04019,928.546429,2014,AZ-501,0.002170,0.001705,0.000465,46233.0,133694.0,149147.0,115392.0,81406.0,592298.0,364938.0,141211.0,993144.0,13.2,2.50,162075.0,149710.0,386155.0,69636.0,57099.0,Arizona,Pima County,1004229.0,494684.0,509545.0,858334.0,41043.0,42683.0,31905.0,2266.0,363063.0,"Pima County, AZ",0.154,"SNAP, Other Nutrition Programs","SNAP, Other Nutrition Programs",2.85,3591.0,838.0,1604.0,463126.0,435183.0,27943.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3371,21999,NaN,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,17.0,16.0,NaN,NaN,NaN,NaN
3372,22999,NaN,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,131.0,0.0,NaN,NaN,NaN,NaN
3373,25999,NaN,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,104.0,3.0,NaN,NaN,NaN,NaN
3374,16999,NaN,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,51.0,NaN,NaN,NaN,NaN


In [23]:
# check to see that number of observations is retained
df.Year.value_counts()

2020    32502
2019    13207
2010     9787
2018     3406
2017     3395
2016     3386
2014     3386
2015     3384
2012     3378
2013     3376
2011     3375
2009     3239
Name: Year, dtype: int64

# Remove Duplicates per County
Some FIPS (counties) are present multiple times with slightly varying observations (likely taken from different sources.) The following process groups the dataframe by FIPS, while taking the mean of the observations, so that there is one row per county.

In [234]:
# Check value counts per FIPS code, indicating there are multiple entries in some cases
df[df.Year=='2010'].FIPS.value_counts()

06037    12
25017    12
17197    12
17031    12
25005     9
         ..
02270     1
26999     1
23999     1
12999     1
02999     1
Name: FIPS, Length: 3245, dtype: int64

In [235]:
# perform groupby's on the years containing duplicates
no_dups_10 = df[df.Year=='2010'].groupby('FIPS').mean().reset_index()
no_dups_19 = df[df.Year=='2019'].groupby('FIPS').mean().reset_index()
no_dups_20 = df[df.Year=='2020'].groupby('FIPS').mean().reset_index()


In [236]:
# re-add in the columns that were removed during groupby operation
no_dups_10 = no_dups_10.merge(df[df.Year=='2010'][['Year','FIPS','Low Threshold Type','High Threshold Type', 'State',
                'coc_number','County','State/County']],on='FIPS').drop_duplicates().dropna(subset=['State/County'])
no_dups_19 = no_dups_19.merge(df[df.Year=='2019'][['Year','FIPS','Low Threshold Type','High Threshold Type', 'State',
                'coc_number','County','State/County']],on='FIPS').drop_duplicates().dropna(subset=['State/County'])
no_dups_20 = no_dups_20.merge(df[df.Year=='2020'][['Year','FIPS','Low Threshold Type','High Threshold Type', 'State',
                'coc_number','County','State/County']],on='FIPS').drop_duplicates().dropna(subset=['State/County'])


In [237]:
# drop older data from main df, which contained duplicates
df_trimmed = df.drop(df[df.Year.isin(['2010', '2019','2020'])].index)

# add newly grouped data with no duplicates back into main df
df = pd.concat([df_trimmed, no_dups_10, no_dups_19, no_dups_20])


In [238]:
df.Year.value_counts()

2018    3406
2017    3395
2016    3386
2014    3386
2015    3384
2019    3282
2010    3279
2009    3239
2020    3219
2012    1785
2013    1785
2011    1785
Name: Year, dtype: int64

# Impute HH Size 2019
Household Size data, collected from the ACS Census API, is drastically higher on average in 2019 than all other years. Other years report an average of 2.52 people per household, while in 2019 the reported average is 13,464.72 people per household. For this reason, 2019 values will be imputed by using the 2018 value for 2019, given that the rate of change year-by-year is negligible. 

### Inspect Yearly Averages

In [239]:
df.groupby('Year')['hh_avg_size'].mean()

Year
2009             NaN
2010        2.515342
2011        2.517657
2012        2.517472
2013        2.524417
2014        2.531881
2015        2.533775
2016        2.533969
2017        2.522340
2018        2.523214
2019    13464.727216
2020             NaN
Name: hh_avg_size, dtype: float64

In [240]:
df.shape

(35331, 46)

### Drop 2019 data and insert 2018 values

In [241]:
df_19 = df[df.Year=='2019']
df_not_19 = df[df.Year!='2019']

In [242]:
df_not_19

,FIPS,Rent,Year,coc_number,Houseless_rate,Sheltered_rate,Unsheltered_rate,hh_med_income,pop_disabled,pop_hs_grad,pop_bachelors,pop_grad_degree,pop_priv_health,pop_public_health,pop_no_health,pop_total,percent_hh_poverty,hh_avg_size,pop_65+,hh_no_vehicle,num_hh,pop_non_citizen,hh_SNAP,State,County,TOT_POP,TOT_MALE,TOT_FEMALE,TOT_WHITE,TOT_BLACK,TOT_NATIVE,TOT_ASIAN,TOT_PACIFIC,TOT_LATINX,State/County,FI Rate,Low Threshold Type,High Threshold Type,Cost Per Meal,Num_wholesale,Num_restaraunts,Num_grocery,Total_workforce,Employed,Unemployed,Unemployment_rate
0,01073,1019.995960,2014,AL-500,0.001396,0.000961,0.000435,45239.0,94584.0,117854.0,81626.0,52774.0,431638.0,211570.0,81336.0,658834.0,14.8,2.48,87036.0,93630.0,259397.0,17519.0,39967.0,Alabama,Jefferson County,659972.0,312491.0,347481.0,355635.0,284082.0,2029.0,10378.0,347.0,24099.0,"Jefferson County, AL",0.197,SNAP,Other Nutrition Program,2.93,483.0,2693.0,400.0,312131.000,292505.000,19626.000,6.3000
1,01117,1229.755051,2014,AL-500,0.001396,0.000961,0.000435,69723.0,22792.0,28911.0,35773.0,18511.0,159655.0,42429.0,19175.0,201168.0,6.2,2.65,23404.0,19762.0,74790.0,7624.0,4706.0,Alabama,Shelby County,206280.0,100304.0,105976.0,174094.0,24247.0,805.0,4403.0,101.0,11872.0,"Shelby County, AL",0.105,SNAP,Other Nutrition Program,3.37,1.0,743.0,2706.0,107208.000,102400.000,4808.000,4.5000
2,04003,1051.250000,2014,AZ-500,0.001469,0.000826,0.000643,45974.0,20341.0,21109.0,12968.0,7566.0,76099.0,50498.0,14868.0,130807.0,13.1,2.47,23593.0,16328.0,48846.0,7947.0,7812.0,Arizona,Cochise County,127314.0,64661.0,62653.0,112238.0,5737.0,2165.0,2757.0,510.0,44374.0,"Cochise County, AZ",0.161,"SNAP, Other Nutrition Programs","SNAP, Other Nutrition Programs",2.81,800.0,72.0,340.0,50969.000,46682.000,4287.000,8.4000
3,04013,1095.670228,2014,AZ-502,0.001318,0.001084,0.000235,53689.0,399455.0,593094.0,490927.0,273108.0,2444443.0,1204681.0,646167.0,3947382.0,12.7,2.74,507428.0,546028.0,1424244.0,373532.0,171581.0,Arizona,Maricopa County,4093648.0,2024659.0,2068989.0,3449404.0,235660.0,112383.0,172425.0,11190.0,1239835.0,"Maricopa County, AZ",0.158,"SNAP, Other Nutrition Programs","SNAP, Other Nutrition Programs",2.90,2389.0,16857.0,6320.0,1961997.000,1848119.000,113878.000,5.8000
4,04019,928.546429,2014,AZ-501,0.002170,0.001705,0.000465,46233.0,133694.0,149147.0,115392.0,81406.0,592298.0,364938.0,141211.0,993144.0,13.2,2.50,162075.0,149710.0,386155.0,69636.0,57099.0,Arizona,Pima County,1004229.0,494684.0,509545.0,858334.0,41043.0,42683.0,31905.0,2266.0,363063.0,"Pima County, AZ",0.154,"SNAP, Other Nutrition Programs","SNAP, Other Nutrition Programs",2.85,3591.0,838.0,1604.0,463126.000,435183.000,27943.000,6.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32452,72145,NaN,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Vega Baja Municipio, PR",NaN,NaN,NaN,NaN,NaN,NaN,NaN,12646.625,11223.375,1423.250,11.2375
32462,72147,NaN,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Vieques Municipio, PR",NaN,NaN,NaN,NaN,NaN,NaN,NaN,2476.875,2252.000,224.875,9.0875
32472,72149,NaN,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Villalba Municipio, PR",NaN,NaN,NaN,NaN,NaN,NaN,NaN,6819.875,6013.500,806.375,11.6500
32482,72151,NaN,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Yabucoa Municipio, PR",NaN,NaN,NaN,NaN,NaN,NaN,NaN,8106.625,7232.250,874.375,10.7500


In [243]:
df_18 = df[df.Year=='2018']
df_19 = df_19.merge(df_18[['FIPS','hh_avg_size']],on='FIPS',how='outer').drop('hh_avg_size_x',axis=1).rename(columns={'hh_avg_size_y':'hh_avg_size'})


In [244]:
df = pd.concat([df_not_19,df_19])

### Save the fully cleaned dataframe using pickle

In [246]:
# save with pickle to preserve datatypes
with open('../pickled/fully_cleaned_data.pickle', "wb") as output_file:
    pickle.dump(df, output_file)